In [1]:
import pandas as pd
import os
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import datetime
from datetime import timedelta
from scipy.stats import pearsonr
from sklearn.manifold import TSNE
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import LeavePGroupsOut
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import metrics
import seaborn as sns
from StudentLife import get_data


In [2]:
u21 = pd.read_csv('u018_all_data.csv')

In [3]:
if 'Unnamed: 0' in u21.columns:
    u21 = u21.drop(['Unnamed: 0'], axis=1)

u21 = u21.dropna(subset=['prev_ema_score'])

In [4]:
u21.shape

(80, 119)

In [5]:
u21

,day,eureka_id,ema_CALM,ema_DEPRESSED,ema_HARM,ema_HOPEFUL,ema_SEEING_THINGS,ema_SLEEPING,ema_SOCIAL,ema_STRESSED,...,audio_silence_avg,audio_silence_std,audio_silence_num_good_days,audio_voice_med_mean,audio_voice_med_mean_normal,audio_voice_med_mean_raw,audio_voice_avg,audio_voice_std,audio_voice_num_good_days,prev_ema_score
0,2015-05-13,u018,3.0,0.0,0.0,3.0,0.0,3.0,3.0,0.0,...,65541.454545,5535.544641,11.0,1205.289583,0.919719,6.476389,5925.181818,2550.754783,11.0,-12.0
1,2015-05-15,u018,3.0,0.0,0.0,3.0,1.0,3.0,3.0,0.0,...,65095.583333,4945.627940,12.0,1119.153472,0.921820,7.504167,6560.166667,2354.743987,12.0,-15.0
2,2015-05-18,u018,3.0,0.0,0.0,3.0,0.0,3.0,3.0,0.0,...,63380.071429,5675.216708,14.0,1372.259028,0.921810,7.975694,7813.928571,4462.029608,14.0,-15.0
3,2015-05-20,u018,3.0,0.0,0.0,3.0,0.0,3.0,3.0,0.0,...,62759.071429,4711.130266,14.0,1322.883333,0.925036,8.063889,8197.428571,4131.779552,14.0,-15.0
4,2015-05-22,u018,3.0,0.0,0.0,3.0,0.0,3.0,3.0,0.0,...,63593.642857,4660.396131,14.0,1322.883333,0.920146,8.441667,8738.571429,3985.743010,14.0,-14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,2016-04-15,u018,3.0,0.0,0.0,3.0,0.0,3.0,3.0,0.0,...,63475.000000,6526.878085,14.0,2222.832639,0.827623,9.811806,9270.428571,3668.667788,14.0,-15.0
76,2016-04-18,u018,3.0,0.0,0.0,3.0,1.0,3.0,3.0,1.0,...,63595.142857,7936.746823,14.0,2222.832639,0.835714,9.081944,8399.714286,4638.250805,14.0,-14.0
77,2016-04-20,u018,3.0,3.0,0.0,3.0,0.0,3.0,3.0,0.0,...,64978.857143,6738.535744,14.0,2134.607639,0.848851,8.204861,7465.642857,3994.010836,14.0,-15.0
78,2016-04-22,u018,3.0,0.0,0.0,3.0,0.0,3.0,3.0,0.0,...,65122.923077,6896.681466,13.0,2189.517708,0.843434,8.505556,7577.000000,4148.023947,13.0,-15.0


In [6]:
for item in u21.columns:
    print(item)

day
eureka_id
ema_CALM
ema_DEPRESSED
ema_HARM
ema_HOPEFUL
ema_SEEING_THINGS
ema_SLEEPING
ema_SOCIAL
ema_STRESSED
ema_THINK
ema_VOICES
ema_neg_score
ema_pos_score
ema_resp_time_median
ema_score
calllog_med_mean
calllog_med_mean_normal
calllog_med_mean_raw
calllog_avg
calllog_std
calllog_num_good_days
sms_in_med_mean
sms_in_med_mean_normal
sms_in_med_mean_raw
sms_in_avg
sms_in_std
sms_in_num_good_days
sms_out_med_mean
sms_out_med_mean_normal
sms_out_med_mean_raw
sms_out_avg
sms_out_std
sms_out_num_good_days
unlock_med_mean
unlock_med_mean_normal
unlock_med_mean_raw
unlock_avg
unlock_std
unlock_num_good_days
conver_med_mean
conver_med_mean_normal
conver_med_mean_raw
conver_avg
conver_std
conver_num_good_days
sleep_med_mean
sleep_med_mean_normal
sleep_med_mean_raw
sleep_avg
sleep_std
sleep_num_good_days
act_invehicle_med_mean
act_invehicle_med_mean_normal
act_invehicle_med_mean_raw
act_invehicle_avg
act_invehicle_std
act_invehicle_num_good_days
act_onbike_med_mean
act_onbike_med_mean_norma

In [7]:
u21['calllog_med_mean_normal']

0     0.866727
1     0.863663
2     0.871025
3     0.883435
4     0.888580
        ...   
75    0.836236
76    0.847767
77    0.849285
78    0.866589
79    0.871357
Name: calllog_med_mean_normal, Length: 80, dtype: float64

In [8]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler

In [10]:
EMA_ITEM = 'ema_score'

In [11]:
X = u21
y = u21[EMA_ITEM]

In [12]:
ema = X

In [13]:
baseline = [c for c in ema.columns if 'avg' in c or 'std' in c]
just_diff = [c for c in ema.columns if 'med_mean' in c and 'raw' not in c]
just_diff_absolute = [c for c in ema.columns if 'med_mean' in c and 'normal' not in c and 'raw' not in c]
just_diff_normal = [c for c in ema.columns if 'med_mean_normal' in c]
raw_fea = [c for c in ema.columns if 'raw' in c]
all_fea = just_diff + baseline 
prev_ema = [c for c in ema.columns if ('prev_' + EMA_ITEM) in c]
all_fea_prev_ema = all_fea + prev_ema
baseline_diff_absolute = baseline + just_diff_absolute
baseline_diff_normal = baseline + just_diff_normal
baseline_diff_raw = baseline + raw_fea
all_fea_raw = all_fea + raw_fea
print('length of baseline should be 34:', len(baseline))
print('length of just_diff_normal should be 17:', len(just_diff_normal))
print('length of baseline_diff_normal should be 51:', len(baseline_diff_normal))


feature_sets = [
    prev_ema,
    prev_ema,
    baseline,
    just_diff_normal,
    baseline_diff_normal,
]
feature_sets_names = [
    'prev_ema',
    'mean_predictor',
    'baseline_mean_std',
    'just_diff_normal',
    'baseline_diff_normal',
]

length of baseline should be 34: 34
length of just_diff_normal should be 17: 17
length of baseline_diff_normal should be 51: 51


In [14]:
selected_fea = baseline

In [15]:
X = X[baseline]

In [16]:
y

0    -15.0
1    -14.0
2    -15.0
3    -15.0
4    -15.0
      ... 
75   -15.0
76   -13.0
77   -12.0
78   -15.0
79   -12.0
Name: ema_score, Length: 80, dtype: float64

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 1/3, random_state = 0, shuffle=False)

In [18]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [19]:
X_train.shape

(53, 34)

In [20]:
X_test.shape

(27, 34)

In [21]:
# from sklearn.linear_model import LinearRegression
# regressor = LinearRegression()
# regressor.fit(X_train, y_train)

In [22]:
# y_pred = regressor.predict(X_test)

In [23]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [24]:
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LinearRegression
# regressor = RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1]).fit(X_train, y_train)
regressor = LinearRegression().fit(X_train, y_train)

In [25]:
regressor.coef_

array([ 1.21420196e+00, -9.46666984e-01, -4.69340117e+00,  4.95397720e+00,
        2.78332695e+00, -3.76062271e+00,  2.68196783e+00,  5.91676770e-01,
       -3.17462696e+00,  3.63442805e+00,  2.70352655e+00, -1.67680969e+00,
        1.99377035e+13, -3.86790763e+00,  1.29938362e+12,  2.54827597e+00,
        3.37076491e+13, -2.13554019e+00,  1.00612691e+14,  3.20708042e+00,
        1.28999048e+13,  1.43461381e+00,  6.27256514e+13, -1.51633083e+00,
        1.16095278e+00,  8.74728380e-01,  8.66010278e-01, -2.58315693e+00,
       -2.03143700e+14, -1.26101041e+00, -2.57208913e+14,  4.67723362e+00,
       -1.42946201e+14, -4.88712360e+00])

In [26]:
y_pred = regressor.predict(X_test)

In [27]:
y_pred_train = regressor.predict(X_train)

In [28]:
# df = pd.DataFrame({'Actual': y_test.values.flatten(), 'Predicted': y_pred.flatten()})
# df

In [29]:
print('Mean Squared Error test linear:', metrics.mean_squared_error(y_test, y_pred))

Mean Squared Error test linear: 66.31017276658731


In [30]:
print('Mean Squared Error train linear:', metrics.mean_squared_error(y_train, y_pred_train))

Mean Squared Error train linear: 4.607138728879668


In [196]:
#ridge

In [31]:
regressor = RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1]).fit(X_train, y_train)

In [32]:
regressor.coef_

array([-0.12835767,  0.2553306 , -0.45945317,  0.94015514, -1.07050012,
       -0.04234891,  0.89271084,  0.45923582, -0.3422836 ,  1.14402779,
        0.73978624, -0.35126222,  2.24502114, -1.3106162 , -0.91904703,
        0.89783212, -1.1978381 ,  0.09851552,  0.31339418,  0.47187653,
       -0.56086088, -0.24838152, -0.43820102, -0.17560459, -0.0744403 ,
       -0.02119272,  0.52142963, -0.87417468, -0.43852446,  0.21225723,
       -0.24193944,  0.48532608,  1.05852734, -0.41838307])

In [33]:
y_pred = regressor.predict(X_test)
y_pred_train = regressor.predict(X_train)

In [34]:
# df = pd.DataFrame({'Actual': y_test.values.flatten(), 'Predicted': y_pred.flatten()})
# df

In [35]:
print('Mean Squared Error test ridge:', metrics.mean_squared_error(y_test, y_pred))
print('Mean Squared Error train ridge:', metrics.mean_squared_error(y_train, y_pred_train))

Mean Squared Error test ridge: 12.192075514566808
Mean Squared Error train ridge: 5.490339513171415


In [36]:
params = {'n_estimators': 300, 'max_depth': 3, 'min_samples_split': 2, 'learning_rate': 0.17} 
regressor = GradientBoostingRegressor(**params).fit(X_train, y_train)

In [37]:
y_pred = regressor.predict(X_test)
y_pred_train = regressor.predict(X_train)

In [38]:
# df = pd.DataFrame({'Actual': y_test.values.flatten(), 'Predicted': y_pred.flatten()})
# df

In [39]:
print('Mean Squared Error test gradient boost:', metrics.mean_squared_error(y_test, y_pred))
print('Mean Squared Error train gradient boost:', metrics.mean_squared_error(y_train, y_pred_train))

Mean Squared Error test gradient boost: 1.5288526763458055
Mean Squared Error train gradient boost: 1.1575375320197759e-16


In [40]:
from sklearn.linear_model import SGDRegressor
regressor = SGDRegressor(penalty='l2', max_iter=1000, random_state=42).fit(X_train, y_train)

In [41]:
y_pred = regressor.predict(X_test)
y_pred_train = regressor.predict(X_train)

In [42]:
print('Mean Squared Error test gradient boost:', metrics.mean_squared_error(y_test, y_pred))
print('Mean Squared Error train gradient boost:', metrics.mean_squared_error(y_train, y_pred_train))

Mean Squared Error test gradient boost: 4.8966824008825105
Mean Squared Error train gradient boost: 5.91291532568307


In [43]:
from sklearn import linear_model
regressor = linear_model.BayesianRidge().fit(X_train, y_train)
y_pred = regressor.predict(X_test)
y_pred_train = regressor.predict(X_train)
print('Mean Squared Error test gradient boost:', metrics.mean_squared_error(y_test, y_pred))
print('Mean Squared Error train gradient boost:', metrics.mean_squared_error(y_train, y_pred_train))

Mean Squared Error test gradient boost: 0.9884548671389314
Mean Squared Error train gradient boost: 8.08593969744532


In [44]:
from sklearn import linear_model
regressor = linear_model.ARDRegression().fit(X_train, y_train)
y_pred = regressor.predict(X_test)
y_pred_train = regressor.predict(X_train)
print('Mean Squared Error test gradient boost:', metrics.mean_squared_error(y_test, y_pred))
print('Mean Squared Error train gradient boost:', metrics.mean_squared_error(y_train, y_pred_train))

Mean Squared Error test gradient boost: 1.9811584142026744
Mean Squared Error train gradient boost: 6.932786138298622


In [45]:
from sklearn.gaussian_process import GaussianProcessRegressor
regressor = GaussianProcessRegressor().fit(X_train, y_train)
y_pred = regressor.predict(X_test)
y_pred_train = regressor.predict(X_train)
print('Mean Squared Error test:', metrics.mean_squared_error(y_test, y_pred))
print('Mean Squared Error train:', metrics.mean_squared_error(y_train, y_pred_train))

print('Mean Absolute Error test:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Absolute Error train:', metrics.mean_absolute_error(y_train, y_pred_train))

Mean Squared Error test: 209.61524335431452
Mean Squared Error train: 1.5569783673888686e-18
Mean Absolute Error test: 14.443964612879357
Mean Absolute Error train: 1.2061886583875814e-09
